In [86]:
import os

dir = '/content/drive/MyDrive/tern_project/Eyal/FinalResults/'

os.chdir(dir)

In [87]:
import configparser

# Initialize the ConfigParser object
config = configparser.ConfigParser()
# Read the INI file
config.read('report_breeding_terns.ini')
# A date to count terns on
date = config.get('General', 'date')
# Path to the output directory for multi-scan tracking results
mult_scans_result_dir = config.get('General', 'mult_scans_result_dir')
# Path to final breeding tracks result
final_breeding_report = config.get('General', 'final_breeding_report')
# Path to overlap areas details
overlap_areas = config.get('General', 'overlap_areas')

### Load breeding tracks and determine classification

collect JSON files list of breeding tracks

In [88]:
import glob


flags_dirs = [os.path.join(mult_scans_result_dir, dir_name) for dir_name in os.listdir(mult_scans_result_dir) if date in dir_name]

# Collect all JSON files
flags_reports_lists = []
for dir_path in flags_dirs:
    # Get all immediate subdirectories
    subdirs = [os.path.join(dir_path, d) for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d))]

    # Search for JSON files in each subdirectory
    for subdir in subdirs:
        flags_reports_lists.extend(glob.glob(os.path.join(subdir, '*.json')))

Create Dataframe of all breeding track

In [89]:
import re

def get_flag_number(file_name):
    # Define a regular expression pattern to match the flag number
    pattern = r'flag(\d+).'
    # Use re.search to find the pattern in the file name
    match = re.search(pattern, file_name)
    # Check if a match is found
    if match:
        # Extract the flag number from the matched group
        return match.group(1)
    else:
        print("No flag number found in the file name.")

In [90]:
from collections import Counter

def select_class(classifier_results):
    # Count the frequency of each class
    class_counts = Counter([result[0] for result in classifier_results])
    max_count = max(class_counts.values())

    # Find classes with the maximum frequency
    most_frequent_classes = [cls for cls, count in class_counts.items() if count == max_count]

    if len(most_frequent_classes) == 1:
        # If only one class is most frequent, return it
        selected_class = most_frequent_classes[0]
        conf = sum([result[1] for result in classifier_results if result[0] == selected_class]) / max_count
    else:
        cls_conf = {}
        for result in classifier_results:
            for cls in most_frequent_classes:
                if cls == result[0]:
                    cls_conf[cls] = cls_conf.get(cls, 0) + result[1]
                    break

        selected_class = max(cls_conf, key=cls_conf.get)
        conf = cls_conf[selected_class] / max_count

    return selected_class, conf

In [91]:
import json

breeding_tracks = []

for flag_file in flags_reports_lists:
    with open(flag_file, 'r') as file:
        tracks_details = json.load(file)

    # Extract flag number and tern classes distribution
    flag_number = get_flag_number(flag_file)


    # Process each nest detail
    for track in tracks_details['nests_details']:
        pred_class, conf = select_class(track['track_classes'])
        relevant_data = {
            **track['average_location'],
            'predict_class': pred_class,
            'confidence': conf,
            'flag': flag_number,
        }
        breeding_tracks.append(relevant_data)

In [92]:
import pandas as pd

# Create dataframe for store boxes details
breeding_tracks_df = pd.DataFrame(breeding_tracks)

Calculating for each box the width and heigh size in cm. The calculations involve camera calibration, 3D to 2D transformations, and the use of PTZ (Pan-Tilt-Zoom) camera parameters. It's done by using a library writted to mapping box pixel coordinates into real world location.

In [93]:
import sys

sys.path.append('/content/drive/MyDrive/tern_project/Eyal/RealCoordinatesCalculator')
from real_coordinates_calculator import RealCoordinatesCalculator


real_coordinates_calculator = RealCoordinatesCalculator()
# Group by 'flag' and apply the box size calculation function
boxes_size_df = breeding_tracks_df.groupby('flag').apply(real_coordinates_calculator.calc_box_size)

# Reset the index for later merging
boxes_size_df.reset_index(drop=True, inplace=True)

# Merge the calculated results back into the original DataFrame
breeding_tracks_df = pd.merge(breeding_tracks_df, boxes_size_df, left_index=True, right_index=True)

# Display the resulting DataFrame
breeding_tracks_df.head()

Loading file /content/drive/MyDrive/tern_project/Eyal//PTZ_modi_Cam_Values_181_mod.txt...
Loading file /content/drive/MyDrive/tern_project/Eyal//PTZ_modi_Cam_Values_191_mod.txt...


<ipython-input-93-006cee63154f>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  boxes_size_df = breeding_tracks_df.groupby('flag').apply(real_coordinates_calculator.calc_box_size)


,x1,x2,y1,y2,predict_class,confidence,flag,tern_x,tern_y,dy/dx_uv,pix_x1,pix_y1,pix_x2,pix_y2,DX_cm,DY_cm,dx_pix_drone,dy_pix_drone,Area
0,460.772936,548.048666,498.667235,552.805161,Common-Sit,0.879668,100,504.410801,552.805161,0.620309,9389,13529,9543,13497,85.583103,53.087975,154,32,4543.433636
1,622.935929,732.560897,304.318416,382.821116,Common-Sit,0.865604,100,677.748413,382.821116,0.716102,8713,13466,9013,13423,166.557746,119.272399,300,43,19865.742040
2,289.365714,385.989536,329.386320,404.944664,Common-Sit,0.903253,99,889.356042,257.859331,0.677526,8227,12909,8550,12901,179.839556,121.845938,323,8,21912.719293
3,294.482485,372.281134,269.702648,329.685423,Common-Sit,0.909961,97,39.238877,308.470894,1.021304,8396,13319,8728,13299,183.980005,187.899537,332,20,34569.757629
4,111.387451,208.217079,533.717542,592.070584,Common-Sit,0.842426,97,830.853707,269.309233,0.621255,8274,12945,8598,12931,180.089921,111.881818,324,14,20148.787807


### Find all terns pairs representations of same tern and remove the one with less prediction confidence. Thus pairs duplication appear because that Yolo often detects the same tern with two boxes- one for each class.

In [94]:
def calc_iou_box_vs_box(box1, box2):
        x1, x1_max, y1, y1_max = box1['x1'], box1['x2'], box1['y1'], box1['y2']
        x2, x2_max, y2, y2_max = box2['x1'], box2['x2'], box2['y1'], box2['y2']

        # Calculating box1 width and height
        w1 = x1_max - x1
        h1 = y1_max - y1

        # Calculating box1 width and height
        w2 = x2_max - x2
        h2 = y2_max - y2

        # Calculating the coordinates of the intersection rectangle
        x_intersection = max(x1, x2)
        y_intersection = max(y1, y2)
        x_intersection_max = min(x1_max, x2_max)
        y_intersection_max = min(y1_max, y2_max)

        # Calculating the area of intersection
        intersection_area = max(0, x_intersection_max - x_intersection) * max(0, y_intersection_max - y_intersection)

        # Calculating the area of union
        box1_area = w1 * h1
        box2_area = w2 * h2
        union_area = box1_area + box2_area - intersection_area

        # Calculating the IoU
        iou = intersection_area / union_area

        return iou

In [95]:
def calculate_iou(box1, box2):
    # Extract coordinates of the bounding boxes
    x1_tl, y1_tl, x1_br, y1_br = box1['x1'], box1['x2'], box1['y1'], box1['y2']
    x2_tl, y2_tl, x2_br, y2_br = box2['x1'], box2['x2'], box2['y1'], box2['y2']
    # print(x1_tl, y1_tl, x1_br, y1_br)
    # Calculate the coordinates of the intersection rectangle
    x_intersection_tl = max(x1_tl, x2_tl)
    y_intersection_tl = max(y1_tl, y2_tl)
    x_intersection_br = min(x1_br, x2_br)
    y_intersection_br = min(y1_br, y2_br)

    # Calculate area of intersection rectangle
    intersection_area = max(0, x_intersection_br - x_intersection_tl + 1) * max(0, y_intersection_br - y_intersection_tl + 1)

    # Calculate area of each bounding box
    area_box1 = (x1_br - x1_tl + 1) * (y1_br - y1_tl + 1)
    area_box2 = (x2_br - x2_tl + 1) * (y2_br - y2_tl + 1)

    # Calculate area of union
    union_area = area_box1 + area_box2 - intersection_area

    # Calculate IoU
    iou = intersection_area / union_area

    return iou

In [96]:
import math

def calc_distance_cm(tern1, tern2):
    i_x_average = (tern1['pix_x2'] + tern1['pix_x1']) / 2
    i_y_average = tern1['pix_y1']
    j_x_average = (tern2['pix_x2'] + tern2['pix_x1']) / 2
    j_y_average = tern2['pix_y1']

    return math.sqrt(pow(abs(i_y_average - j_y_average), 2) + pow(abs(i_x_average - j_x_average), 2))

def find_duplicate_terns(sub_df, threshold=0.5):
    """
    Find all pairs of rectangles with IoU greater than the threshold.
    """
    duplicate_pairs = []
    flag = sub_df.iloc[0]['flag']
    for i in range(len(sub_df)):
        for j in range(i + 1, len(sub_df)):
            iou = calc_iou_box_vs_box(sub_df.iloc[i], sub_df.iloc[j])
            if iou > threshold:
                # distance = calc_distance_cm(sub_df.iloc[i], sub_df.iloc[j])
                duplicate_pairs.append((i, j))
                # duplicate_pairs.append((flag, i, j, iou, distance))
    return duplicate_pairs

In [97]:
duplicate_pairs = breeding_tracks_df.groupby('flag').apply(find_duplicate_terns)

<ipython-input-97-1bf94ae1083f>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  duplicate_pairs = breeding_tracks_df.groupby('flag').apply(find_duplicate_terns)


In [98]:
duplicate_pairs

,0
flag,
100,[]
101,[]
102,"[(5, 7)]"
103,"[(0, 18), (1, 9), (16, 19)]"
104,"[(4, 13), (8, 28), (11, 12), (21, 24)]"
...,...
95,[]
96,[]
97,[]


Keep one of each duplicate pair of the same tern. We keep the one tern with higher prediction probability.

In [99]:
duplicates_to_remove = []

for flag, pairs in duplicate_pairs.items():
    if len(pairs) == 0:
        continue

    # get index of first row of current flag
    flag_indx = breeding_tracks_df[breeding_tracks_df['flag'] == flag].index[0]

    for pair in pairs:
        confidence1 = breeding_tracks_df.iloc[flag_indx + pair[0]]['confidence']
        confidence2 = breeding_tracks_df.iloc[flag_indx + pair[1]]['confidence']
        duplicate_tern_indx = flag_indx + (pair[0] if confidence1 > confidence2 else pair[1])
        duplicates_to_remove.append(duplicate_tern_indx)


# Remove duplicate terns based on indexes
breeding_tracks_df = breeding_tracks_df.drop(duplicates_to_remove)
breeding_tracks_df = breeding_tracks_df.reset_index(drop=True)

print(f'{len(duplicates_to_remove)} duplicate terns found and removed..')

27 duplicate terns found and removed..


Remove duplicate terns from different flags due to flags overlay.

In [100]:
import json

# Read the JSON file
with open(overlap_areas, 'r') as json_file:
    redundant_areas = json.load(json_file)

In [101]:
def is_point_in_polygon(point, polygon):
  """
  Determines if a point is inside a polygon.

  Args:
    point: A tuple (x, y) representing the point.
    polygon: A list of tuples [(x1, y1), (x2, y2), ...] representing the polygon vertices.

  Returns:
    True if the point is inside the polygon, False otherwise.
  """

  x, y = point
  inside = False

  for i in range(len(polygon)):
    p1x, p1y = polygon[i]
    p2x, p2y = polygon[(i + 1) % len(polygon)]
    if y > min(p1y, p2y):
      if y <= max(p1y, p2y):
        if x <= max(p1x, p2x):
          if p1y != p2y:
            x_inters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
            if p1x == p2x or x <= x_inters:
              inside = not inside

  return inside

In [102]:
def is_point_in_polygon1(point, polygon):
    """Check if a point (x, y) is inside a given polygon."""
    n = len(polygon)
    inside = False

    px, py = point
    for i in range(n):
        j = (i + 1) % n
        ix, iy = polygon[i]
        jx, jy = polygon[j]

        if ((iy > py) != (jy > py)) and (px < (jx - ix) * (py - iy) / (jy - iy) + ix):
            inside = not inside

    return inside

from shapely.geometry import Point, Polygon

def is_point_in_polygon2(point, polygon):
    # return False
    return point.within(polygon)

count = 0
def is_point_in_any_polygon(point, polygons):
    global count
    """Check if a point (x, y) is inside any of the given polygons."""
    for polygon in polygons:
        count += 1
        if is_point_in_polygon2(point, Polygon(polygon)):
            # print(point)
            # print(polygon)
            return True
    return False

In [104]:
from shapely.geometry import Point, Polygon


# Collect indices of rows to remove
indices_to_remove = []

for index, track in breeding_tracks_df.iterrows():
    if(track['flag'] not in redundant_areas):
        continue
    center = Point((track.loc['pix_y1'] + track.loc['pix_y2']) / 2, (track.loc['pix_x1'] + track.loc['pix_x2']) / 2)
    if is_point_in_any_polygon(center, redundant_areas[track['flag']]):
        indices_to_remove.append(index)

breeding_tracks_df = breeding_tracks_df.drop(index=indices_to_remove)
breeding_tracks_df = breeding_tracks_df.reset_index(drop=True)


#### Display the terns locations on the drone map

In [106]:
import cv2

## The drone image
drone_img = cv2.imread('/content/drive/MyDrive/tern_project/drone_rotem/Shafiyot_True_Ortho.tif')

In [107]:
from google.colab.patches import cv2_imshow

shelved_flags = ['120', '121', '137', '138']

for index, row in breeding_tracks_df.iterrows():
    if row['flag'] in shelved_flags:
        continue
    if 'Common' in row['predict_class']:
        cv2.circle(drone_img, [row['pix_x1'], row['pix_y1']], radius=25, color=(0,0,255), thickness=-1)
    elif 'Little' in row['predict_class']:
        cv2.circle(drone_img, [row['pix_x1'], row['pix_y1']], radius=25, color=(0,255,255), thickness=-1)
    else:
        cv2.circle(drone_img, [row['pix_x1'], row['pix_y1']], radius=25, color=(255,0,0), thickness=-1)


# Resize the image
scale_factor = 0.1
resized_image = cv2.resize(drone_img, (0, 0), fx=scale_factor, fy=scale_factor)

cv2_imshow(resized_image)

Output hidden; open in https://colab.research.google.com to view.

In [108]:
breeding_tracks_counter = Counter(breeding_tracks_df['predict_class'])

print(f"Terns on colony on {date}:")
for category, count in breeding_tracks_counter.items():
    print(f"{category}: {count}")

Terns on colony on 2024_06_11:
Common-Sit: 666
Common-Stand: 101
Little-Sit: 109
Background: 10
Chick: 3


Save breeding terns report

In [75]:
# Ensure the directory exists
os.makedirs(f"{final_breeding_report}/{date}", exist_ok=True)
print(f"Saving report on {date}")
# Group by 'flag' and generate JSON files
for flag, group in breeding_tracks_df.groupby("flag"):
    report = group[["pix_x1", "pix_y1", "predict_class"]].to_dict(orient="records")
    file_path = f"{final_breeding_report}/{date}/flag{flag}.json"

    with open(file_path, "w") as json_file:
        json.dump(report, json_file, indent=4)

print(f"All reports are saved...")

Saving report on 2024_06_11
All reports saved...


In [67]:
breeding_tracks_df.columns

Index(['x1', 'x2', 'y1', 'y2', 'predict_class', 'flag', 'tern_x', 'tern_y',
       'dy/dx_uv', 'pix_x1', 'pix_y1', 'pix_x2', 'pix_y2', 'DX_cm', 'DY_cm',
       'dx_pix_drone', 'dy_pix_drone', 'Area'],
      dtype='object')